In [1]:
# ============================================================
# AI Anomaly Detection on Gold Layer (SQL Server)
# ============================================================

import pyodbc
import pandas as pd
from sklearn.ensemble import IsolationForest

print("Starting AI Anomaly Detection...")

# ------------------------------------------------------------
# 1. CONNECT TO SQL SERVER
# ------------------------------------------------------------
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=LAPTOP-IJ8NES98\\SQLEXPRESS;"
    "DATABASE=SecurityLogsDW;"
    "Trusted_Connection=yes;"
)

print("Connected to SecurityLogsDW")

# ------------------------------------------------------------
# 2. LOAD GOLD TABLES
# ------------------------------------------------------------
print("Loading Gold tables...")

df_ip = pd.read_sql("SELECT * FROM gold.ip_risk_summary", conn)
df_time = pd.read_sql("SELECT * FROM gold.login_time_summary", conn)
df_geo = pd.read_sql("SELECT * FROM gold.geo_risk_summary", conn)
df_device = pd.read_sql("SELECT * FROM gold.device_risk_summary", conn)
df_session = pd.read_sql("SELECT * FROM gold.intrusion_session_summary", conn)

print("Gold tables loaded")

# ------------------------------------------------------------
# 3. GENERIC ANOMALY DETECTION FUNCTION
# ------------------------------------------------------------
def detect_anomalies(df, feature_columns, contamination=0.05):
    model = IsolationForest(
        n_estimators=100,
        contamination=contamination,
        random_state=42
    )
    
    df["anomaly_flag"] = model.fit_predict(df[feature_columns])
    df["is_anomaly"] = df["anomaly_flag"].apply(lambda x: 1 if x == -1 else 0)
    
    return df

# ------------------------------------------------------------
# 4. APPLY AI ON EACH GOLD TABLE
# ------------------------------------------------------------

# 4.1 IP-Level Anomalies
print("Running IP-level anomaly detection...")
df_ip = detect_anomalies(
    df_ip,
    ["failure_rate", "attack_attempts", "total_attempts"]
)

# 4.2 Time-Based Anomalies
print("Running time-based anomaly detection...")
df_time = detect_anomalies(
    df_time,
    ["total_logins", "failed_logins", "attack_logins", "failure_rate"]
)

# 4.3 Geography-Based Anomalies
print("Running geo-based anomaly detection...")
df_geo = detect_anomalies(
    df_geo,
    ["total_logins", "failed_logins", "attack_logins", "failure_rate"]
)

# 4.4 Device / Browser Anomalies
print("Running device/browser anomaly detection...")
df_device = detect_anomalies(
    df_device,
    ["total_logins", "failed_logins", "attack_logins", "failure_rate"]
)

# 4.5 Session-Level Anomalies
print("Running session-level anomaly detection...")
df_session = detect_anomalies(
    df_session,
    ["avg_session_duration", "avg_failed_logins", "detected_attacks", "high_risk_sessions"]
)

# ------------------------------------------------------------
# 5. SAVE AI RESULTS (CSV OUTPUT)
# ------------------------------------------------------------
df_ip.to_csv("ip_anomaly_results.csv", index=False)
df_time.to_csv("time_anomaly_results.csv", index=False)
df_geo.to_csv("geo_anomaly_results.csv", index=False)
df_device.to_csv("device_anomaly_results.csv", index=False)
df_session.to_csv("session_anomaly_results.csv", index=False)

print("Anomaly results saved as CSV files")

# ------------------------------------------------------------
# 6. SUMMARY OUTPUT
# ------------------------------------------------------------
print("============================================")
print("AI ANOMALY DETECTION COMPLETED SUCCESSFULLY")
print("============================================")

print(f"Suspicious IPs detected: {df_ip['is_anomaly'].sum()}")
print(f"Suspicious time windows: {df_time['is_anomaly'].sum()}")
print(f"Suspicious regions: {df_geo['is_anomaly'].sum()}")
print(f"Suspicious devices/browsers: {df_device['is_anomaly'].sum()}")
print(f"Suspicious sessions: {df_session['is_anomaly'].sum()}")

Starting AI Anomaly Detection...


OperationalError: ('08001', '[08001] [Microsoft][ODBC Driver 17 for SQL Server]Shared Memory Provider: Could not open a connection to SQL Server [2].  (2) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 17 for SQL Server]A network-related or instance-specific error has occurred while establishing a connection to SQL Server. Server is not found or not accessible. Check if instance name is correct and if SQL Server is configured to allow remote connections. For more information see SQL Server Books Online. (2)')